In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("c:/users/user/Downloads/film/Financial-Portfolio-Optimization-and-Time-Series-Forecasting/src/")
from modules import (data_preparation_for_model, model_builder, forecasting_utils, portfolio_optimizer, visualization_utils)

In [2]:
# Step 1: Load and preprocess data
file_path = "../data/tesla.csv"
columns=["Adj Close","Close","High","Low","Open","Volume","Daily Change","Volatility"]
tesla,scaler  = data_preparation_for_model.load_and_scale_data(file_path,columns)
X_train, y_train, X_test, y_test = data_preparation_for_model.prepare_train_test_data(tesla)

In [3]:

X_train = X_train[:, :, 1:].astype(np.float64)
y_train = y_train[:, 1:].astype(np.float64)
X_test = X_test[:, :, 1:].astype(np.float64)
y_test = y_test[:, 1:].astype(np.float64)

In [4]:
X_train.dtype
y_train.dtype
X_test.dtype
y_test.dtype

dtype('float64')

In [5]:
# Step 2: Build, train, and evaluate LSTM model
input_shape = (1, X_train.shape[2])
model = model_builder.build_lstm_model(input_shape)
model = model_builder.train_lstm_model(model, X_train, y_train)


C:\Users\User\Downloads\film\Financial-Portfolio-Optimization-and-Time-Series-Forecasting\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - loss: 0.0548
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.0316
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0324
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0327
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.0318
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0315
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0321
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.0316
Epoch 9/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.0322
Epoch 10/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.0317
Epoch 11/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0315
Epoch 12/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0318
Epoch 13/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0315
Epoch 14/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0321
Epoch 15/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0318
Epoc

In [7]:
# Step 3: Make predictions and visualize
y_pred = forecasting_utils.make_predictions(model, X_test, scaler)
y_pred = y_pred.reshape(-1, 8)
y_test = y_test.reshape(-1, 8)
y_test = scaler.inverse_transform(y_test)
visualization_utils.plot_predictions(df, int(len(df) * 0.8), y_pred, y_test)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


ValueError: cannot reshape array of size 434 into shape (8)

In [ ]:
# Step 4: Forecast future data
last_sequence = X_test[-1]
future_predictions = forecasting_utils.forecast_future(model, last_sequence, future_steps=180, scaler=scaler)
future_dates = pd.date_range(df.index[-1], periods=180, freq='B')
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
visualization_utils.plot_future_forecast(df, int(len(df) * 0.8), y_pred, future_predictions, future_dates, rmse)


In [ ]:
# Step 5: Portfolio Optimization
# Load forecasts for other assets and optimize portfolio
# For example: future_df = pd.DataFrame({'TSLA': future_predictions.flatten(), 'BND': bnd_forecast, 'SPY': spy_forecast})
# mean_returns, cov_matrix = portfolio_optimizer.calculate_returns_and_covariance(future_df)
# optimal_weights = portfolio_optimizer.optimize_portfolio(mean_returns, cov_matrix, num_assets=3)
# print("Optimal Weights:", optimal_weights)